### Train a machine learning model 

#### Step 1: Setup 

In [86]:
#%pip install pandas numpy -qU

In [87]:
#%pip install scikit-learn -qU

In [88]:
# %pip install shap -qU

In [89]:
 #%pip install xgboost -qU

In [90]:
# %pip install  matplotlib -qU

In [91]:
 #%pip install numpy -qU

In [92]:
# %pip install catboost -qU

In [93]:
# %pip install lightgbm -qU

In [94]:
# %pip install seaborn -qU

In [95]:
# %pip install optuna -qU

In [96]:
# !nvidia-smi

In [97]:
import pandas as pd
import os
import sklearn
import shap
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.model_selection import train_test_split, cross_validate,RandomizedSearchCV, StratifiedKFold
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
import matplotlib.pyplot as plt
from trainMLmodel import surfaceDesignTopAndBottom,selectDescriptors,combineRankAndDescriptorsClassyfication,explainer, combineRankAndDescriptorsClassificationbotisFLAT, combineRankAndDescriptorsClassyfication_Danilingus
import numpy as np

In [98]:
cd

/bme001.mnt/home/20212358


#### Step 2: load the data file

In [99]:
# Define the location of the csv files imageOutliersRemoved and Ranking
#pathToImageObject=os.getcwd()+"/DataAnalysis/"
pathToImageObject="/bme001.mnt/home/20212358"
locationOfImageObject1="cleaned_rawImagedataFI_final.csv"
locationOfImageObject2="DataAnalysis_withJan/Screen_Mean_Count_CellBody_new.csv"
# Define the location of the annotation files
pathToTopoUnitImages="/FeatureImages/FeatureImages/"
allTopoDescriptors=pd.read_csv("allTopoDescriptors.csv")
# check if file exists
if os.path.isfile(locationOfImageObject1):
    imagedataOutliersRemoved=pd.read_csv(locationOfImageObject1,low_memory=False)
    if os.path.isfile(locationOfImageObject2):
        imageDataRankedSurfaces=pd.read_csv(locationOfImageObject2,low_memory=False)
    else:
        print("File not found, check if you finished ranking the features")
else:
    print('File not find, check if you removed the outliers')


In [100]:
#imagedataOutliersRemoved["Mean_Nephrin_Granularity_8_CorrCellBody"]

In [101]:
#replacing ranking by previous module with ranking sorted from lowest hit to the highest
#locationOfImageObject2=pathToImageObject+"/DataAnalysis/Screen_Mean_Mean_Nephrin_AreaShape_MajorAxisLength.csv"
# locationOfImageObject2=pathToImageObject+"/DataAnalysis/PostBlindTestHits/Screen_Mean_Mean_Nephrin_Granularity_8_CorrCellBody.csv"
# imageDataRankedSurfaces=pd.read_csv(locationOfImageObject2,low_memory=False)

In [102]:
imageDataRankedSurfaces

FeatureIdx  Screen_ADpvalue_Count_CellBody_new  \
0            710                                0.25   
1           1877                                0.25   
2           1448                                0.25   
3           1076                                0.25   
4           1054                                0.25   
...          ...                                 ...   
1829        1288                                0.25   
1830        1742                                0.25   
1831         836                                0.25   
1832        1292                                0.25   
1833        1625                                0.25   

      Screen_KSpvalue_Count_CellBody_new  Screen_SNR_Count_CellBody_new  \
0                               0.474026                       3.799227   
1                               0.930736                       5.293229   
2                               0.962704                       3.913095   
3                               0.962704                       3.102444   
4                               0.660140                       2.805779   
...                                  ...                            ...   
1829                            0.637363                       2.343262   
1830                            0.660140                       2.257996   
1831                            0.735664                       2.389780   
1832                            0.316239                       2.175057   
1833                            1.000000                       2.795554   

      Screen_Mean_Count_CellBody_new  Screen_SD_Count_CellBody_new  \
0                          47.833333                     12.590280   
1                          47.583333                      8.989472   
2                          46.785714                     11.956193   
3                          46.428571                     14.965161   
4                          46.125000                     16.439282   
...                              ...                           ...   
1829                       24.571429                     10.485993   
1830                       24.466667                     10.835568   
1831                       24.000000                     10.042766   
1832                       23.384615                     10.751267   
1833                       19.833333                      7.094599   

      Screen_Median_Count_CellBody_new    Rank  
0                                 49.5     1.0  
1                                 47.5     2.0  
2                                 49.5     3.0  
3                                 53.5     4.0  
4                                 53.0     5.0  
...                                ...     ...  
1829                              23.0  1830.0  
1830                              24.0  1831.0  
1831                              21.0  1832.0  
1832                              22.0  1833.0  
1833                              18.5  1834.0  

[1834 rows x 8 columns]

In [103]:
# locationOfImageObject3=pathToImageObject+"/DataAnalysis/Screen_Mean_Count_CellBody_new_Ranking.csv"
# imageDataRankedCount=pd.read_csv(locationOfImageObject3,low_memory=False)
# imageDataRankedCount

#### Step 3: Select Features for the machine learning.

##### Step 3.1 Vizualize the topographies design from top and bottom of the rank.

First, select the feature of interest

In [104]:
featureOfInterest='Screen_Mean_Count_FilteredNuclei_new'

Second, select the number of top and bottom ranked feautures you want to select

In [105]:
numberOfRankedFeaturesToVisualize=100

In [106]:
#surfaceDesignTopAndBottom(imageDataRankedSurfaces,pathToTopoUnitImages,featureOfInterest,number=numberOfRankedFeaturesToVisualize)
imagedataOutliersRemoved

AreaOccupied_AreaOccupied_CellBody_new  \
0                                    195441.0   
1                                    190993.0   
2                                    157466.0   
3                                    128276.0   
4                                    159374.0   
...                                       ...   
31915                                146459.0   
31916                                113561.0   
31917                                136466.0   
31918                                108352.0   
31919                                132796.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_new  \
0                                           41149.0   
1                                           34146.0   
2                                           21593.0   
3                                           30686.0   
4                                           36365.0   
...                                             ...   
31915                                       28944.0   
31916                                       21965.0   
31917                                       22229.0   
31918                                       18080.0   
31919                                       35925.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_target  \
0                                              38157.0   
1                                              33484.0   
2                                              19752.0   
3                                              28584.0   
4                                              35031.0   
...                                                ...   
31915                                          24587.0   
31916                                          19472.0   
31917                                          18386.0   
31918                                          12223.0   
31919                                          27669.0   

       AreaOccupied_AreaOccupied_MaskedNuclei  \
0                                    147590.0   
1                                    178264.0   
2                                    112684.0   
3                                    115514.0   
4                                    147877.0   
...                                       ...   
31915                                128723.0   
31916                                 96719.0   
31917                                109961.0   
31918                                 73657.0   
31919                                106458.0   

       AreaOccupied_AreaOccupied_Nephrin  AreaOccupied_Perimeter_CellBody_new  \
0                               166158.0                              17129.0   
1                               210194.0                              18661.0   
2                               133496.0                              13267.0   
3                               138323.0                              11218.0   
4                               200466.0                              16275.0   
...                                  ...                                  ...   
31915                           165853.0                              13291.0   
31916                           153661.0                              10378.0   
31917                           143216.0                              12602.0   
31918                           146310.0                              10716.0   
31919                           133671.0                              10817.0   

       AreaOccupied_Perimeter_FilteredNuclei_new  \
0                                         6242.0   
1                                         5837.0   
2                                         3797.0   
3                                         4632.0   
4                                         5380.0   
...                                          ...   
31915                                     4030.0   
31916                                     3448.0   
31917                         

### Step 4 Load and vizualize descriptors data

##### Step 4.1 select descriptors based on a keyword 

In [107]:
selectedDescriptors= selectDescriptors(allDescriptors=allTopoDescriptors,
             # keywordsToInclude=["Kamiel"],
                 # keywordsToExclude=["_Index","Height","Width", "Center","Texture","ImageNumber","Zernike","sd","Count","Number","var","AngleBetweenNeighbors","mad","Neighbors"])
#selectedDescriptors= selectDescriptors(allDescriptors=allTopoDescriptors,
              #  keywordsToInclude=["Pattern","Kamiel"],
                 #keywordsToExclude=["_Index","Height","Width", "Center","Number","Zernike"])
   keywordsToExclude=["_Index","Height","Width", "Center","Number","Zernike","Texture"])

In [108]:
pd.options.display.max_columns = 30
selectedDescriptors

FeatureIdx  Pattern_AreaOccupied_AreaOccupied_Pattern  \
0              1                                       5215   
1              2                                       2604   
2              3                                      20366   
3              4                                       4864   
4              5                                      17246   
...          ...                                        ...   
2171        2172                                       3821   
2172        2173                                      16244   
2173        2174                                       1855   
2174        2175                                       4794   
2175        2176                                      23548   

      Pattern_AreaOccupied_Perimeter_Pattern  \
0                                        330   
1                                        422   
2                                        910   
3                                        439   
4                                       1252   
...                                      ...   
2171                                     436   
2172                                    1497   
2173                                     306   
2174                                     492   
2175                                    1632   

      Pattern_AreaOccupied_TotalArea_Pattern  Pattern_Count_Pattern  \
0                                      40000                      1   
1                                      10000                      1   
2                                      40000                      1   
3                                      10000                      1   
4                                      40000                      1   
...                                      ...                    ...   
2171                                   10000                      1   
2172                                   78400                      2   
2173                                   10000                      1   
2174                                   10000                      2   
2175                                   78400                      1   

      Pattern_AreaShape_Area_sd  Pattern_AreaShape_Compactness_sd  \
0                         0.000                          0.000000   
1                         0.000                          0.000000   
2                         0.000                          0.000000   
3                         0.000                          0.000000   
4                         0.000                          0.000000   
...                         ...                               ...   
2171                      0.000                          0.000000   
2172                   3183.395                          0.654076   
2173                      0.000                          0.000000   
2174                   1681.500                          0.306309   
2175                      0.000                          0.000000   

      Pattern_AreaShape_Eccentricity_sd  Pattern_AreaShape_Extent_sd  \
0                              0.000000                     0.000000   
1                              0.000000                     0.000000   
2                              0.000000                     0.000000   
3                              0.000000                     0.000000   
4                              0.000000                     0.000000   
...                                 ...                          ...   
2171                           0.000000                     0.000000   
2172                           0.052437                     0.022692   
2173                           0.000000                     0.000000   
2174                           0.553171                     0.158168   
2175                           0.000000                     0.000000   

      Pattern_AreaShape_FormFactor_sd  Pattern_AreaShape_MajorAxisLength_sd  \
0                            0.000000         

In [109]:
Screen_Mean_Mean_Nephrin_Granularity_1 = pd.read_csv("Screen_Mean_Mean_Nephrin_Granularity_1_CorrCellBody.csv")
Screen_Mean_Mean_Nephrin_Granularity_1

FeatureIdx  Screen_ADpvalue_Mean_Nephrin_Granularity_1_CorrCellBody  \
0           1658                                           0.250000         
1            189                                           0.083959         
2           1270                                           0.095665         
3            122                                           0.250000         
4           1353                                           0.250000         
...          ...                                                ...         
2086        1836                                           0.163023         
2087        1860                                           0.250000         
2088        1394                                           0.250000         
2089         757                                           0.069970         
2090        1292                                           0.250000         

      Screen_KSpvalue_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              0.730111         
1                                              0.126748         
2                                              0.055944         
3                                              0.647552         
4                                              0.700123         
...                                                 ...         
2086                                           0.078568         
2087                                           0.860428         
2088                                           0.730111         
2089                                           0.078568         
2090                                           0.893706         

      Screen_SNR_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              5.508653    
1                                              5.717872    
2                                              4.253026    
3                                              5.190216    
4                                              4.866214    
...                                                 ...    
2086                                           5.657096    
2087                                           5.332652    
2088                                           5.167565    
2089                                           6.501527    
2090                                           3.432248    

      Screen_Mean_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                             46.026999     
1                                             45.273450     
2                                             45.259933     
3                                             45.068389     
4                                             44.444154     
...                                                 ...     
2086                                          30.269535     
2087                                          29.868709     
2088                                          29.775861     
2089                                          29.053096     
2090                                          28.751350     

      Screen_SD_Mean_Nephrin_Granularity_1_CorrCellBody  \
0                                              8.355400   
1                                              7.917885   
2                                             10.641819   
3                                              8.683335   
4                                              9.133211   
...                                                 ...   
2086                                           5.350720   
2087                                           5.601099   
2088                                           5.762068   
2089                                           4.468657   
2090                                           8.376829   

      Screen_Median_Mean_Nephrin_Granularity_1_CorrCellBody    Rank  
0                                             45.283829         1.0

### Step 5: Train Machine Learning models

##### Step 5.1 : train the model based on the ranked surfaces and the descriptors selected in step 4.1

In [110]:
imageDataRankedSurfaces

FeatureIdx  Screen_ADpvalue_Count_CellBody_new  \
0            710                                0.25   
1           1877                                0.25   
2           1448                                0.25   
3           1076                                0.25   
4           1054                                0.25   
...          ...                                 ...   
1829        1288                                0.25   
1830        1742                                0.25   
1831         836                                0.25   
1832        1292                                0.25   
1833        1625                                0.25   

      Screen_KSpvalue_Count_CellBody_new  Screen_SNR_Count_CellBody_new  \
0                               0.474026                       3.799227   
1                               0.930736                       5.293229   
2                               0.962704                       3.913095   
3                               0.962704                       3.102444   
4                               0.660140                       2.805779   
...                                  ...                            ...   
1829                            0.637363                       2.343262   
1830                            0.660140                       2.257996   
1831                            0.735664                       2.389780   
1832                            0.316239                       2.175057   
1833                            1.000000                       2.795554   

      Screen_Mean_Count_CellBody_new  Screen_SD_Count_CellBody_new  \
0                          47.833333                     12.590280   
1                          47.583333                      8.989472   
2                          46.785714                     11.956193   
3                          46.428571                     14.965161   
4                          46.125000                     16.439282   
...                              ...                           ...   
1829                       24.571429                     10.485993   
1830                       24.466667                     10.835568   
1831                       24.000000                     10.042766   
1832                       23.384615                     10.751267   
1833                       19.833333                      7.094599   

      Screen_Median_Count_CellBody_new    Rank  
0                                 49.5     1.0  
1                                 47.5     2.0  
2                                 49.5     3.0  
3                                 53.5     4.0  
4                                 53.0     5.0  
...                                ...     ...  
1829                              23.0  1830.0  
1830                              24.0  1831.0  
1831                              21.0  1832.0  
1832                              22.0  1833.0  
1833                              18.5  1834.0  

[1834 rows x 8 columns]

In [111]:
imageDataRankedSurfaces.shape

(1834, 8)

In [112]:
# #pd.merge(left=imageDataRankedSurfaces, right=result_df,
#                       how='left', left_on="FeatureIdx", 
#                       right_on="FeatureIdx").Rank_x

In [113]:
X,Y=combineRankAndDescriptorsClassyfication(SortedRank=imageDataRankedSurfaces,Descriptors=selectedDescriptors,numberPerClass=90)

X.shape

(180, 383)

In [114]:
Y=Y.astype(bool)

In [115]:
def reduce_memory_usage(df):
    import gc
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()
                
                if (df[col].nunique()==2):
                    df[col] = df[col].astype(bool)

                elif str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)

                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
            else:
                if (df[col].nunique()==2):
                    df[col] = df[col].astype(bool)
                    
                if(df[col].nunique()>100):
                    df[col] = df[col].astype('category')
                else:
                    if((col_type != 'datetime64[ns]')):
                        df[col] = df[col].astype(str)
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage became: ",mem_usg," MB")
    gc.collect()
    return df

X=reduce_memory_usage(X)

Memory usage of dataframe is 0.53 MB
Memory usage became:  0.13556289672851562  MB


##### Step 5.2: split X and Y in training and test data set

In [116]:
a=0
b=0
for i in Y.values:
    if (i==False):
        b+=1
    else:
        a+=1
a,b

(90, 90)

In [117]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,stratify=Y )
# X_train,X_val,y_train,y_val=train_test_split(X_train,y_train)

In [118]:
X_test,X_val,y_test,y_val=train_test_split(X_test,y_test,test_size=0.2,stratify=y_test )


##### Step 5.3: select a classification model

In [119]:
import lightgbm as lgb
import joblib
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score


def objective(trial,X_train =X_train ,y_train = y_train, X_test = X_test,y_test=y_test, big_df = X, big_y=Y ):

    param = {
    'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart','rf']), #, 'rf' 
    'num_leaves': trial.suggest_int('num_leaves', 2, 2500),
    'max_depth':trial.suggest_int('max_depth', -10, 900),
    'objective':trial.suggest_categorical('objective', ['binary']),
    'learning_rate': trial.suggest_float('learning_rate', 1e-12, 0.99, log=False),
    'n_estimators': trial.suggest_int('n_estimators', 100, 24_100),
    #'subsample_for_bin': trial.suggest_int('subsample_for_bin', 50_000, 450_000, 50_000),
    'random_state': trial.suggest_categorical('random_state', [42]),
    'data_sample_strategy':trial.suggest_categorical('data_sample_strategy', ['bagging','goss']),
    # 'tree_learner':trial.suggest_categorical('tree_learner', ['serial', 'feature', 'data', 'voting']),
    'deterministic':trial.suggest_categorical('deterministic', [True]),
    'force_col_wise':trial.suggest_categorical('force_col_wise', [True,False]),
    # 'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 2_230),
    
    'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf',  0, 100),
# #     'bagging_fraction': trial.suggest_float('bagging_fraction',  1e-12, 1),
    'feature_fraction': trial.suggest_float('feature_fraction', 1e-12, 0.9, log=False),
    'feature_fraction_bynode': trial.suggest_float('feature_fraction_bynode', 1e-12, 0.9, log=False),
    'extra_trees':trial.suggest_categorical('extra_trees', [True,False]),
    'lambda_l1': trial.suggest_float('lambda_l1',  0, 100),
    'lambda_l2': trial.suggest_float('lambda_l2',  0, 100),
    'linear_lambda': trial.suggest_float('linear_lambda',  0, 100),
    'drop_rate': trial.suggest_float('drop_rate',  0, 1),
    'max_drop': trial.suggest_int('max_drop', 0, 2000),
    'skip_drop': trial.suggest_float('skip_drop',  0, 1),
    'uniform_drop':trial.suggest_categorical('uniform_drop', [True,False]),
    'top_rate': trial.suggest_float('top_rate',  0, 0.5),
    'other_rate': trial.suggest_float('other_rate',  0, 0.5),
    'min_data_per_group': trial.suggest_int('min_data_per_group', 1, 2000),
    'max_cat_threshold': trial.suggest_int('max_cat_threshold', 1, 2000),
    'cat_l2': trial.suggest_float('cat_l2',  0, 100),
    'cat_smooth': trial.suggest_float('cat_smooth',  0, 100),
    'top_k': trial.suggest_int('top_k', 1, 2000),
    'cegb_tradeoff': trial.suggest_float('cegb_tradeoff',  0, 100),
    # 'cegb_penalty_split': trial.suggest_float('cegb_penalty_split',  0, 100),
    # 'path_smooth': trial.suggest_float('path_smooth',  0, 100),
    # 'num_grad_quant_bins': trial.suggest_int('num_grad_quant_bins', 1, 2000),
#     'is_unbalance':trial.suggest_categorical('uniform_drop', [True,False]),
    'scale_pos_weight': trial.suggest_float('scale_pos_weight',  1e-12, 100),
    'sigmoid': trial.suggest_float('sigmoid',  1e-12, 100),
    # 'boost_from_average':trial.suggest_categorical('boost_from_average', [True,False]),
    'max_bin':trial.suggest_int('max_bin', 50, 1_000),
        
    'n_jobs':trial.suggest_categorical('n_jobs', [-1]), 
    'verbose': trial.suggest_categorical('verbose', [-1]), 
    }


    model = lgb.LGBMClassifier(**param)
    model.fit(X_train, y_train, eval_set=[
              (X_test,y_test)])


    y_pred_test = model.predict(X_test)
    # print(y_pred_test)
    

    #accuracy_test = balanced_accuracy_score(y_pred_test, y_test)

    #accuracy_test = (sklearn.model_selection.cross_val_score (model, big_df, big_y, cv=5, scoring='balanced_accuracy', n_jobs=1)).mean()
    # try:
    #     accuracy_test = roc_auc_score(y_pred_test, y_test)
    # except Exception:
    #     accuracy_test=0

    try:
        predicted_train = model.predict(X_train)
        probabilities_train = model.predict_proba(X_train)
        predicted_test = model.predict(X_test)
        probabilities_test = model.predict_proba(X_test)
            
        train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:,1])
        test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:,1])
    
        accuracy_train = metrics.auc(train_fpr, train_tpr)
        accuracy_test = metrics.auc(test_fpr, test_tpr)

        bal_accuracy_train = balanced_accuracy_score(predicted_train, y_train)
        bal_accuracy_test = balanced_accuracy_score(predicted_test, y_test) 
    except Exception:
        accuracy_test=0
        accuracy_train = 0
        bal_accuracy_test = 0
        bal_accuracy_train=0


    joblib.dump(study, 'another_lgbm_with_val.pkl')

    if (int(trial.number)%50==0):
        print("BACKUP GOES BRRRRR")
        joblib.dump(study, 'another_lgbm_with_val_BACKUP.pkl')


    return accuracy_test, accuracy_train,bal_accuracy_train,bal_accuracy_test

In [120]:
# https://vaex.io/docs/tutorial_ml.html#CatBoost-example

In [121]:
def logging_callback(study, frozen_trial):
    # Retrieve previous best values for both objectives
    previous_best_value_0 = study.user_attrs.get("previous_best_value_0", None)
    previous_best_value_1 = study.user_attrs.get("previous_best_value_1", None)
    previous_best_value_2 = study.user_attrs.get("previous_best_value_2", None)
    previous_best_value_3 = study.user_attrs.get("previous_best_value_3", None)

    
    # Get the current trial's values
    current_values = frozen_trial.values

    if current_values is None:
        return  # Skip if the trial doesn't have valid results

    # current_value_0, current_value_1 = current_values
    current_value_0, current_value_1,current_value_2,current_value_3 = current_values


    # Check if both objectives improved
    if (
        (previous_best_value_0 is None or current_value_0 >= previous_best_value_0) and
        (previous_best_value_1 is None or current_value_1 >= previous_best_value_1) and
        (previous_best_value_2 is None or current_value_2 >= previous_best_value_2) and
        (previous_best_value_3 is None or current_value_3 >= previous_best_value_3) 
    ):
        # Update the best values in the study's user attributes
        study.set_user_attr("previous_best_value_0", current_value_0)
        study.set_user_attr("previous_best_value_1", current_value_1)
        study.set_user_attr("previous_best_value_2", current_value_2)
        study.set_user_attr("previous_best_value_3", current_value_3)

        # Log the improved metrics
        print(40*"*",)
        print("\n\n\n")
        print(
            f"\nNew best trial with both objectives improved!\n"
            f"Trial {frozen_trial.number}:\n"
            f"  Objective 0: {current_value_0}\n"
            f"  Objective 1: {current_value_1}\n"
            f"  Objective 2: {current_value_2}\n"
            f"  Objective 3: {current_value_3}\n"
            f"  Params: {frozen_trial.params}\n"
        )
        print("\n\n\n")

        print(40*"*","\n")


In [122]:
#!rm another_lgbm_with_val.pkl

In [38]:
!free -th

              total        used        free      shared  buff/cache   available
Mem:          377Gi       168Gi       186Gi       2.3Gi        21Gi       202Gi
Swap:         3.7Gi       3.7Gi       0.0Ki
Total:        381Gi       172Gi       186Gi


In [91]:
import shap
import matplotlib.pyplot as plt
import joblib
import lightgbm as lgb

# Load the Optuna study
study = joblib.load('another_lgbm_with_val.pkl')  # Ensure the correct path
plt.close('all')

# Retrieve the best trial from the Pareto front
best_trial = study.best_trials[2]  # Select the first Pareto-optimal trial
best_params = best_trial.params

print("Best Parameters:", best_params)

# Load the dataset (ensure these variables are defined in your notebook)
# Assuming X_train, y_train, X_test are already defined
# X_train = ...
# y_train = ...
# X_test = ...

# Train the LightGBM model with the best parameters
model = lgb.LGBMClassifier(**best_params)  # Use LGBMRegressor if regression
model.fit(X_train, y_train)

# Use TreeExplainer for LightGBM models
explainer = shap.TreeExplainer(model)

# Compute SHAP values
shap_values = explainer.shap_values(X_test)

# Plot and save the SHAP summary plot with Arial font and size 11
shap.summary_plot(shap_values, X_test, show=False)

# Get current axes
ax = plt.gca()

# Modify font properties for labels, title, and tick labels
ax.set_xlabel(ax.get_xlabel(), fontname='Arial', fontsize=11)
ax.set_ylabel(ax.get_ylabel(), fontname='Arial', fontsize=11)
ax.set_title(ax.get_title(), fontname='Arial', fontsize=11)
ax.tick_params(axis='x', labelsize=11)
ax.tick_params(axis='y', labelsize=11)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontname('Arial')
    item.set_fontsize(11)

plt.savefig("shap_summary_plot_arial.png", bbox_inches='tight')
print("SHAP summary plot saved as 'shap_summary_plot_arial.png'")

Best Parameters: {'boosting_type': 'dart', 'num_leaves': 2363, 'max_depth': 653, 'objective': 'binary', 'learning_rate': 0.26475907552680067, 'n_estimators': 11478, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': False, 'min_sum_hessian_in_leaf': 68.44220566510118, 'feature_fraction': 0.4494306255621695, 'feature_fraction_bynode': 0.5228226077689093, 'extra_trees': False, 'lambda_l1': 46.1365202165305, 'lambda_l2': 28.73693088321542, 'linear_lambda': 35.37467962491439, 'drop_rate': 0.01103140386183257, 'max_drop': 1738, 'skip_drop': 0.7200478442801592, 'uniform_drop': True, 'top_rate': 0.11784933384924705, 'other_rate': 0.29566451699252067, 'min_data_per_group': 1061, 'max_cat_threshold': 1421, 'cat_l2': 27.915802098383036, 'cat_smooth': 34.56586476187168, 'top_k': 1772, 'cegb_tradeoff': 2.6063011190894345, 'scale_pos_weight': 30.442069101334383, 'sigmoid': 49.80889096689197, 'max_bin': 703, 'n_jobs': -1, 'verbose': -1}


/home/20212358/.local/lib/python3.10/site-packages/shap/explainers/_tree.py:448: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn('LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray')
findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.


SHAP summary plot saved as 'shap_summary_plot_arial.png'


In [92]:
import joblib
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier, plot_importance

# Load Optuna study
study = joblib.load('another_lgbm_with_val.pkl')
plt.close('all')

# Get the best trial parameters
best_trial = study.best_trials[2]
best_params = best_trial.params

# Train the model with the best parameters
bestClassyficationModel = LGBMClassifier(**best_params)
bestClassyficationModel.fit(X_train, y_train)

# Plot feature importance with Arial font and size 11
fig, ax = plt.subplots(figsize=(10, 10))
plot_importance(bestClassyficationModel, max_num_features=15, ax=ax, importance_type='split')

# Modify font properties for labels, title, and tick labels
ax.set_xlabel(ax.get_xlabel(), fontname='Arial', fontsize=11)
ax.set_ylabel(ax.get_ylabel(), fontname='Arial', fontsize=11)
ax.set_title("Top 15 Features by Importance", fontname='Arial', fontsize=11)
ax.tick_params(axis='x', labelsize=11)
ax.tick_params(axis='y', labelsize=11)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontname('Arial')
    item.set_fontsize(11)

plt.savefig("Top 15 Features by Importance Arial.png", bbox_inches='tight') # changed the name to avoid overwrite
plt.show() # show the plot

In [124]:
%%time 


#optuna.logging.set_verbosity(optuna.logging.WARNING)

import optuna, gc
gc.collect()

if os.path.isfile('another_lgbm_with_val.pkl'):
    print("Found file !")
    study = joblib.load('another_lgbm_with_val.pkl')
else:
    print("CREATING NEW FILE...")
    sampler = optuna.samplers.TPESampler(multivariate=True, warn_independent_sampling=False)
    study = optuna.create_study(directions=['maximize', 'maximize','maximize','maximize'], load_if_exists=True,sampler=sampler,pruner=optuna.pruners.MedianPruner(n_warmup_steps=30,n_startup_trials=20))
study.optimize(objective, timeout=1_000_000, gc_after_trial=True,
              show_progress_bar=True, n_jobs=1, callbacks=[logging_callback])
print("Finished working")
joblib.dump(study, 'another_lgbm_with_val.pkl')

Found file !


   0%|          | 00:00/277:46:40

[I 2025-04-02 10:29:16,240] Trial 511 finished with values: [0.5918367346938775, 0.55883487654321, 0.5192513368983958, 0.5933333333333333] and parameters: {'boosting_type': 'dart', 'num_leaves': 32, 'max_depth': 599, 'objective': 'binary', 'learning_rate': 0.8872223213534175, 'n_estimators': 5071, 'random_state': 42, 'data_sample_strategy': 'bagging', 'deterministic': True, 'force_col_wise': False, 'min_sum_hessian_in_leaf': 86.56568463184261, 'feature_fraction': 0.27327754269224014, 'feature_fraction_bynode': 0.12265901920042806, 'extra_trees': True, 'lambda_l1': 55.336493547884665, 'lambda_l2': 2.697707836965101, 'linear_lambda': 1.738756982435493, 'drop_rate': 0.3919078851667688, 'max_drop': 1263, 'skip_drop': 0.8398342223915056, 'uniform_drop': True, 'top_rate': 0.1929483883880502, 'other_rate': 0.31436475106883177, 'min_data_per_group': 1923, 'max_cat_threshold': 1179, 'cat_l2': 99.32540440923106, 'cat_smooth': 57.71624891424206, 'top_k': 583, 'cegb_tradeoff': 16.14671815650419, '

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[I 2025-04-02 10:29:37,736] Trial 515 finished with values: [0.5, 0.5, 0.5, 0.5] and parameters: {'boosting_type': 'rf', 'num_leaves': 2407, 'max_depth': 898, 'objective': 'binary', 'learning_rate': 0.855540274166544, 'n_estimators': 19843, 'random_state': 42, 'data_sample_strategy': 'bagging', 'deterministic': True, 'force_col_wise': True, 'min_sum_hessian_in_leaf': 91.38891696648159, 'feature_fraction': 0.5709605407303169, 'feature_fraction_bynode': 0.7094028619979609, 'extra_trees': True, 'lambda_l1': 52.624236425482714, 'lambda_l2': 47.555864203563175, 'linear_lambda': 88.56686540822746, 'drop_rate': 0.04258289476022148, 'max_drop': 626, 'skip_drop': 0.0845028885668887, 'uniform_drop': True, 'top_rate': 0.414139475237193, 'other_rate': 0.22439159242354925, 'min_data_per_group': 1916, 'max_cat_threshold': 841, 'cat_l2': 10.607232333080539, 'cat_smooth': 49.3453137886162, 'top_k': 1919, 'cegb_tradeoff': 6.6961204267845105, 'scale_pos_weight': 87.63849879517842, 'sigmoid': 96.43805902

...

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[I 2025-04-02 10:49:32,899] Trial 679 finished with values: [0.5, 0.5, 0.5, 0.5] and parameters: {'boosting_type': 'dart', 'num_leaves': 1724, 'max_depth': 734, 'objective': 'binary', 'learning_rate': 0.38750137939012735, 'n_estimators': 23765, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': True, 'min_sum_hessian_in_leaf': 22.187623772789223, 'feature_fraction': 0.47331987601016856, 'feature_fraction_bynode': 0.7969189858350627, 'extra_trees': True, 'lambda_l1': 89.95820012098542, 'lambda_l2': 38.99682593986272, 'linear_lambda': 13.310735690876804, 'drop_rate': 0.27931242764840586, 'max_drop': 248, 'skip_drop': 0.35768910476189986, 'uniform_drop': True, 'top_rate': 0.39896942225233367, 'other_rate': 0.21793429372462364, 'min_data_per_group': 637, 'max_cat_threshold': 432, 'cat_l2': 90.45844007046438, 'cat_smooth': 99.69349314723092, 'top_k': 512, 'cegb_tradeoff': 68.84562660700666, 'scale_pos_weight': 81.07946162570282, 'sigmoid': 9.3127904

/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/20212358/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[I 2025-04-02 10:49:35,719] Trial 680 finished with values: [0.5, 0.5, 0.5, 0.5] and parameters: {'boosting_type': 'gbdt', 'num_leaves': 253, 'max_depth': 896, 'objective': 'binary', 'learning_rate': 0.29550104171316444, 'n_estimators': 18813, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': False, 'min_sum_hessian_in_leaf': 17.55232112604932, 'feature_fraction': 0.44102666767989684, 'feature_fraction_bynode': 0.09837895283938569, 'extra_trees': True, 'lambda_l1': 56.04941011452045, 'lambda_l2': 57.21398003257075, 'linear_lambda': 38.92652504860918, 'drop_rate': 0.17946829658683572, 'max_drop': 282, 'skip_drop': 0.9490816067437434, 'uniform_drop': True, 'top_rate': 0.0005290552525824554, 'other_rate': 0.1030322319289107, 'min_data_per_group': 497, 'max_cat_threshold': 1063, 'cat_l2': 62.445438080486724, 'cat_smooth': 52.13217688403192, 'top_k': 1678, 'cegb_tradeoff': 74.69930255661299, 'scale_pos_weight': 73.32633672088475, 'sigmoid': 90.4726

KeyboardInterrupt: 

In [ ]:
print(1/0)
#CHECK TRIAL 1580, 1709, 1845, 4669, 9338

#after second december 2024
#check 643 (0.17,0.94);  765 (0.16, 1.0);  801 (0.154, 1.0);

In [125]:
study = joblib.load('another_lgbm_with_val.pkl')



In [126]:
# trial_number = 4669
# specific_trial = next((t for t in study.trials if t.number == trial_number), None)

# if specific_trial is not None:
#     print(f"Trial #{specific_trial.number}")
#     print("Parameters:", specific_trial.params)
#     print("Value:", specific_trial.values)
# else:
#     print(f"Trial #{trial_number} not found.")

In [127]:
print('Number of finished trials:', len(study.trials))
try:
    print('Best trial:', study.best_trial.params)
except Exception:
    pass

Number of finished trials: 681


In [128]:
try:
    Best_trial = study.best_trial.params
    #Best_trial['task_type']='GPU'
    Best_trial['verbose'] = -1
    print(Best_trial)
except Exception:
    Best_trial = study.best_trials[0].params
    #Best_trial['task_type']='GPU'
    Best_trial['verbose'] = -1
    print(Best_trial)

{'boosting_type': 'gbdt', 'num_leaves': 191, 'max_depth': 118, 'objective': 'binary', 'learning_rate': 0.01679910379082482, 'n_estimators': 12042, 'random_state': 42, 'data_sample_strategy': 'goss', 'deterministic': True, 'force_col_wise': True, 'min_sum_hessian_in_leaf': 87.74013625036233, 'feature_fraction': 0.25455318721130743, 'feature_fraction_bynode': 0.375789890524636, 'extra_trees': False, 'lambda_l1': 14.825534539789798, 'lambda_l2': 95.37157659523345, 'linear_lambda': 70.86772503160189, 'drop_rate': 0.73844801734339, 'max_drop': 1267, 'skip_drop': 0.2862573476293778, 'uniform_drop': False, 'top_rate': 0.17018468861347003, 'other_rate': 0.09885757333923684, 'min_data_per_group': 435, 'max_cat_threshold': 1734, 'cat_l2': 85.84302114069115, 'cat_smooth': 62.80892158399032, 'top_k': 887, 'cegb_tradeoff': 98.43099413468349, 'scale_pos_weight': 4.6257547692635566, 'sigmoid': 64.16478915867698, 'max_bin': 387, 'n_jobs': -1, 'verbose': -1}


In [129]:
for i in range(len(study.best_trials)):
    print(study.best_trials[i].values)

[0.8673469387755102, 0.9226466049382716, 0.7589928057553956, 0.5]
[0.8316326530612246, 0.8967978395061729, 0.7757446808510639, 0.8368983957219251]
[0.7908163265306122, 1.0, 1.0, 0.8181818181818181]
[0.8418367346938775, 0.9359567901234567, 0.8269565217391304, 0.7625]
[0.854591836734694, 0.7934027777777777, 0.7660098522167488, 0.7592592592592593]
[0.8214285714285714, 0.8952546296296297, 0.7911051212938005, 0.8368983957219251]
[0.836734693877551, 0.892554012345679, 0.7859531772575251, 0.8111111111111111]
[0.8520408163265307, 0.8859953703703703, 0.78125, 0.7592592592592593]
[0.8163265306122449, 0.8892746913580247, 0.7757446808510639, 0.8645833333333333]
[0.8418367346938775, 0.8927469135802469, 0.7775974025974026, 0.7865497076023391]
[0.8367346938775511, 0.8983410493827161, 0.7709030100334449, 0.7865497076023391]
[0.826530612244898, 0.9006558641975309, 0.7886928628708901, 0.7865497076023391]
[0.836734693877551, 0.8775077160493827, 0.8046683046683047, 0.7865497076023391]
[0.8367346938775511,

In [45]:
try:
    print(study.best_trial.value)
except Exception:
    print(study.best_trials[2].values)

[0.7857142857142858, 0.9984567901234568, 0.9864864864864865, 0.7512820512820513]


###### study.best_trials[1].params

In [138]:
%%capture

"""
try:
    best_model = lgb.LGBMClassifier(**study.best_params)
    best_model.fit(X_train, y_train, eval_set=[
                  (X_test,y_test)])
except Exception:
    print("Using alternative")
    best_model = lgb.LGBMClassifier(**study.best_trials[0].params)
    best_model.fit(X_train, y_train, eval_set=[
                  (X_test,y_test)])

"""

best_model = lgb.LGBMClassifier(**study.best_trials[1].params)
#best_model = lgb.LGBMClassifier(**specific_trial.params)
best_model.fit(X_train, y_train, eval_set=[
              (X_test,y_test)])

In [139]:
# best_model = cb.CatBoostClassifier(**study.best_params,
#                                 ).fit(X, Y, verbose=False, early_stopping_rounds=500)

In [140]:
best_model.predict(X_test)

array([ True, False,  True,  True, False,  True, False,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False, False, False, False,  True,
       False])

In [141]:
predicted_train = best_model.predict(X_train)
probabilities_train = best_model.predict_proba(X_train)
predicted_test = best_model.predict(X_test)
probabilities_test = best_model.predict_proba(X_test)

#FOR VALIDATION
predicted_val = best_model.predict(X_val)
probabilities_val = best_model.predict_proba(X_val)

In [142]:
y_test.nunique()

2

In [143]:
from sklearn.metrics import balanced_accuracy_score

display(balanced_accuracy_score(y_test, predicted_test))
display(balanced_accuracy_score(y_train, predicted_train))

0.8214285714285714

0.75

In [144]:
# FOR VALIDATION

display(balanced_accuracy_score(y_val, predicted_val))

0.75

In [145]:
import matplotlib.pyplot as plt
from sklearn import metrics

# Assuming y_train, probabilities_train, y_test, probabilities_test are defined
plt.close('all')
# Calculate the false and true positive rates (fpr, tpr)
train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:, 1])
test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:, 1])

# Obtain the values for the AUC plot
train_roc_auc = metrics.auc(train_fpr, train_tpr)
test_roc_auc = metrics.auc(test_fpr, test_tpr)
print(train_roc_auc, test_roc_auc)

# Make the plot with Arial font and size 11
fig, ax = plt.subplots(figsize=[5, 5])
ax.set_title('Receiver Operating Characteristic', fontname='Arial', fontsize=11)
ax.plot(train_fpr, train_tpr, 'b', label='Train AUC = %0.2f' % train_roc_auc)
ax.plot(test_fpr, test_tpr, 'g', label='Test AUC = %0.2f' % test_roc_auc)
ax.legend(loc='lower right', prop={'family': 'Arial', 'size': 11})
ax.plot([0, 1], [0, 1], 'r--')
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.set_ylabel('True Positive Rate', fontname='Arial', fontsize=11)
ax.set_xlabel('False Positive Rate', fontname='Arial', fontsize=11)
ax.tick_params(axis='x', labelsize=11)
ax.tick_params(axis='y', labelsize=11)

for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontname('Arial')
    item.set_fontsize(11)

plt.savefig('another_lgbm_with_val_arial.png')  # Change the filename to avoid overwrite
plt.show() # show the plot

0.8967978395061729 0.8316326530612246


In [133]:
#-------------------

In [134]:
#together with Koen
bestClassyficationModel=RandomForestClassifier(**bestParameters)
bestClassyficationModel.fit(X_train, y_train)

NameError: name 'RandomForestClassifier' is not defined

In [135]:
predicted_train = bestClassyficationModel.predict(X_train)
probabilities_train = bestClassyficationModel.predict_proba(X_train)
predicted_test = bestClassyficationModel.predict(X_test)
probabilities_test = bestClassyficationModel.predict_proba(X_test)

NameError: name 'bestClassyficationModel' is not defined

Step 5.7 :Print out accuracy metrics

In [ ]:
print ('\nTrain Accuracy:', metrics.accuracy_score(y_train, predicted_train))
print ('Test Accuracy:', metrics.accuracy_score(y_test, predicted_test))


Step 5.8: plot the AUC curve for training and test data

In [ ]:
# calculate the false and true positive rates (fpr, tpr)
train_fpr, train_tpr, train_threshold = metrics.roc_curve(y_train, probabilities_train[:,1])
test_fpr, test_tpr, test_threshold = metrics.roc_curve(y_test, probabilities_test[:,1])
#obtain the values for the AUC plot
train_roc_auc = metrics.auc(train_fpr, train_tpr)
test_roc_auc = metrics.auc(test_fpr, test_tpr)
#make the plot
fig, ax = plt.subplots(figsize=[5,5])
plt.title('Receiver Operating Characteristic')
plt.plot(train_fpr, train_tpr, 'b', label = 'Train AUC = %0.2f' % train_roc_auc)
plt.plot(test_fpr, test_tpr, 'g', label = 'Test AUC = %0.2f' % test_roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Step 5.9: Plot 10 most important features for the model

In [ ]:
plot_importance(bestClassyficationModel,max_num_features=20,xlabel='Features weight');    
plt.show()

### Step 6: Use SHAP module to explain predicted model

##### Step 6.1: Show which features contribute to prediction of particular surface

In [ ]:
explainerDefinition=explainer(dataForModel=X,
          model=bestClassyficationModel,
          hit=1)

##### Step 6.2: Show SHAP values for every important feature and every predicted surface Plot explanation https://shap.readthedocs.io/en/latest/

In [ ]:
shap_values = explainerDefinition.shap_values(X)
shap.summary_plot(shap_values, X)

##### Step 6.3: Draw a scatter plot with features of interest

Set the feature of interest for the x- and y-axis

In [ ]:
xFeature="Kamiel_FCP"
yFeature='InsCirclRadius_median'

In [ ]:
fig, ax = plt.subplots(figsize=[10,10])

colors = {0:'orange', 1:'blue'}

binned=X
binned["Class"]=Y

grouped = binned.groupby("Class")

for key, group in grouped:
    group.plot(ax=ax, kind='scatter', 
               x=xFeature, 
               y=yFeature, 
               label=key, color=colors[key])

plt.show()

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

# 1. Handle Class Imbalance with SMOTE
smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 2. Scale Features
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

# 3. Hyperparameter Search Space Refinement
searchParameters = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [5, 10, 20],
    'learning_rate': [0.0001, 0.001, 0.01],
    'n_estimators': [100, 250, 500],
    'reg_alpha': [0.01, 0.1, 1],
    'reg_lambda': [0.01, 0.1, 1]
}

# 4. Updated Fitting Parameters
fittingParameters = {
    "early_stopping_rounds": 50,
    "eval_metric": "auc",
    "eval_set": [(X_test_scaled, y_test)]  # Using validation set for early stopping
}

# 5. Classification Model
classificationModel = XGBClassifier(objective='binary:logistic', random_state=1)

# 6. Randomized Search with Cross-Validation
random_search = RandomizedSearchCV(
    estimator=classificationModel,
    param_distributions=searchParameters,
    n_iter=50,  # Reduce iterations for quicker testing; increase later
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=10,
    random_state=1
)

# 7. Fit the Model
searchModel = random_search.fit(X_resampled_scaled, y_resampled, **fittingParameters)

# 8. Best Parameters
bestParameters = searchModel.best_estimator_.get_params()
print("Best Parameters:", bestParameters)

# 9. Evaluate Model on Test Data
y_pred = searchModel.predict(X_test_scaled)
test_auc = roc_auc_score(y_test, y_pred)
train_auc = roc_auc_score(y_resampled, searchModel.predict(X_resampled_scaled))

print(f"Train AUC: {train_auc}")
print(f"Test AUC: {test_auc}")
